In [ ]:
###SYSTEM INIT.
import pygame, os, sys, time, random
pygame.init()
pygame.font.init()
pygame.mixer.init()

WIDTH, HEIGHT = 1280, 720
WIN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Galactic Front")



###PROPS INIT.
#LOAD SHIP IMAGE
ALIEN_1 = pygame.transform.scale(pygame.image.load("alien_1.png"),(70,55))
ALIEN_2 = pygame.transform.scale(pygame.image.load("alien_2.png"),(70,55))
ALIEN_3 = pygame.transform.scale(pygame.image.load("alien_3.png"),(70,55))
SPACE_SHIP_1 = pygame.transform.scale(pygame.image.load("spaceship4.png"),(70,60))
SPACE_SHIP_2 = pygame.transform.scale(pygame.image.load("spaceship5.png"),(70,60))
#LOAD LASER IMAGE
LASER_1 = pygame.transform.scale(pygame.image.load("laser_1.png"),(42,52))
LASER_2 = pygame.transform.scale(pygame.image.load("laser_2a.png"),(45,50))
LASER_3 = pygame.transform.scale(pygame.image.load("laser_3.png"),(45,55))
LASER_4 = pygame.transform.scale(pygame.image.load("laser_4a.png"),(35,50))
#LOAD BACKGROUND
BG = pygame.transform.scale(pygame.image.load("background.png"), (WIDTH, HEIGHT))
#LOAD SOUND
crash_1 = pygame.mixer.Sound("crash_1.wav")
crash_2 = pygame.mixer.Sound("crash_2.wav")
crash_3 = pygame.mixer.Sound("crash_3.wav")
crash_list = [crash_1, crash_2, crash_3]
expl_1 = pygame.mixer.Sound("expl_1.wav")
expl_2 = pygame.mixer.Sound("expl_2.wav")
expl_3 = pygame.mixer.Sound("expl_3.wav")
expl_list = [expl_1, expl_2, expl_3]
lasound_1 = pygame.mixer.Sound("lasound_1.wav")
lasound_2 = pygame.mixer.Sound("lasound_2.wav")
hit = pygame.mixer.Sound("hit.wav")
alarm = pygame.mixer.Sound("alarm.wav")
warp = pygame.mixer.Sound("warp.wav")
level_up = pygame.mixer.Sound("level_up.wav")
button_sound = pygame.mixer.Sound("button.wav")



###GAME INIT.
score = 0
multiplayer = False
player10 = True
player20 = True
player_laser_cooldown = 0.8



###CLASSES
class Laser:
    def __init__(self, x, y, img):
        self.x = x
        self.y = y
        self.img = img
        self.mask = pygame.mask.from_surface(self.img)

    def draw(self, window):
        window.blit(self.img, (self.x, self.y))

    def move(self, vel):
        self.y += vel

    def off_screen(self, height):
        return not(self.y <= height and self.y >= -50)

    def collision(self, obj):
        return collide(self, obj)

class Ship:
    COOLDOWN = 25

    def __init__(self, x, y, health=100):
        self.x = x
        self.y = y
        self.health = health
        self.ship_img = None
        self.laser_img = None
        self.lasers = []
        self.cool_down_counter = 0

    def draw(self, window):
        window.blit(self.ship_img, (self.x, self.y))
        for laser in self.lasers:
            laser.draw(window)

    def move_lasers(self, vel, obj):
        self.cooldown()
        for laser in self.lasers:
            laser.move(vel)
            if laser.off_screen(HEIGHT):
                self.lasers.remove(laser)
            elif laser.collision(obj):
                obj.health -= 5
                pygame.mixer.Sound.play(hit)
                self.lasers.remove(laser)

    def cooldown(self):
        if self.cool_down_counter >= self.COOLDOWN:
            self.cool_down_counter = 0
        elif self.cool_down_counter > 0:
            self.cool_down_counter += 1

    def shoot(self):
        if self.cool_down_counter == 0:
            laser = Laser(self.x+15, self.y-45, self.laser_img)
            self.lasers.append(laser)
            pygame.mixer.Sound.play(lasound_1)
            self.cool_down_counter = player_laser_cooldown

    def get_width(self):
        return self.ship_img.get_width()

    def get_height(self):
        return self.ship_img.get_height()
    
class Player(Ship):
    def __init__(self, x, y, health=100):
        super().__init__(x, y, health) 
        self.ship_img = SPACE_SHIP_1
        self.laser_img = LASER_4
        self.mask = pygame.mask.from_surface(self.ship_img)
        self.max_health = health

    def move_lasers(self, vel, objs):
        global score
        self.cooldown()
        for laser in self.lasers:
            laser.move(vel)
            if laser.off_screen(HEIGHT+50):
                self.lasers.remove(laser)
            else:
                for obj in objs:
                    if laser.collision(obj):
                        objs.remove(obj)
                        score += 1
                        random_expl = random.choice(expl_list)
                        pygame.mixer.Sound.play(random_expl)
                        if laser in self.lasers:
                            self.lasers.remove(laser)

    def draw(self, window):
        super().draw(window)
        self.healthbar(window)

    def healthbar(self, window):
        pygame.draw.rect(window, (255,0,0), ((self.x+5), self.y + self.ship_img.get_height() + 7, (self.ship_img.get_width()-10), 10))
        pygame.draw.rect(window, (0,255,0), ((self.x+5), self.y + self.ship_img.get_height() + 7, (self.ship_img.get_width()-10)* (self.health/self.max_health), 10))

class Player2(Ship): 
    def __init__(self, x, y, health=100):
        super().__init__(x, y, health) 
        self.ship_img = SPACE_SHIP_2
        self.laser_img = LASER_4
        self.mask = pygame.mask.from_surface(self.ship_img)
        self.max_health = health

    def move_lasers(self, vel, objs):
        global score_2
        self.cooldown()
        for laser in self.lasers:
            laser.move(vel)
            if laser.off_screen(HEIGHT+50):
                self.lasers.remove(laser)
            else:
                for obj in objs:
                    if laser.collision(obj):
                        objs.remove(obj)
                        score_2 += 1
                        random_expl = random.choice(expl_list)
                        pygame.mixer.Sound.play(random_expl)
                        if laser in self.lasers:
                            self.lasers.remove(laser)

    def draw(self, window):
        super().draw(window)
        self.healthbar(window)

    def healthbar(self, window):
        pygame.draw.rect(window, (255,0,0), ((self.x+5), self.y + self.ship_img.get_height() + 7, (self.ship_img.get_width()-10), 10))
        pygame.draw.rect(window, (0,255,0), ((self.x+5), self.y + self.ship_img.get_height() + 7, (self.ship_img.get_width()-10) * (self.health/self.max_health), 10))
        
class Enemy(Ship):
    MAP = {
                "red": (ALIEN_1, LASER_1),
                "green": (ALIEN_2, LASER_2),
                "blue": (ALIEN_3, LASER_3)
                }

    def __init__(self, x, y, color, health=100):
        super().__init__(x, y, health)
        self.ship_img, self.laser_img = self.MAP[color]
        self.mask = pygame.mask.from_surface(self.ship_img)

    def move(self, vel):
        self.y += vel

    def shoot(self):
        if self.cool_down_counter == 0:
            laser = Laser(self.x+9, self.y+10, self.laser_img)
            self.lasers.append(laser)
            self.cool_down_counter = 100
    
    
    
#COLLISION (enemy, player)            
def collide(obj1, obj2):
    offset_x = obj2.x - obj1.x
    offset_y = obj2.y - obj1.y
    return obj1.mask.overlap(obj2.mask, (offset_x, offset_y)) != None



#MAIN PROGRAM/THE GAME
def main():
    run = True
    
    #CONSTANTS
    FPS = 60
    level = 0
    lives = 5
    enemies = []
    wave_length = 5
    enemy_vel = 1
    player_vel = 5
    laser_vel = 6
    
    global score
    global score_2
    score = 0
    score_2 = 0
    
    #INITIALIZE PLAYERS
    if multiplayer:
        player1 = Player(285, 550)
        player2 = Player2(945, 550)  
    else:
        player = Player(605, 550)
           
    main_font = pygame.font.Font("Game of Squids.ttf", 30)
    sub_font = pygame.font.Font("Expansiva.ttf",25)
    sub_font_2 = pygame.font.Font("Expansiva.ttf",20)
         
    clock = pygame.time.Clock()
    lost = False
    lost_count = 0

    def redraw_window():
        WIN.blit(BG, (0,0))
        
        #LIVES,COUNTER,GAME-TYPE IN-GAME BAR
        if multiplayer:
            mode_label = main_font.render(f"MULTIPLAYER", 1, (255,255,255))
            total = sub_font_2.render(f"P1 Score:  {score}", 1, (255,255,255))
            total_2 = sub_font_2.render(f"P2 Score:  {score_2}", 1, (255,255,255))
            lives_label = sub_font_2.render(f"Chances: {lives}", 1, (255,255,255))
            if score_2 > 9:
                WIN.blit(total_2, (WIDTH - mode_label.get_width() - 10, 100))
            else:
                WIN.blit(total_2, (WIDTH - mode_label.get_width() - 0, 100))
            
            if player10:
                player1.draw(WIN)
            if player20:
                player2.draw(WIN)
        else:
            mode_label = main_font.render(f"SINGLEPLAYER", 1, (255,255,255))
            total = sub_font_2.render(f"Score: {score}", 1, (255,255,255))
            lives_label = sub_font_2.render(f"Chances: {lives}", 1, (255,255,255))
            player.draw(WIN) 
        
        level_label = sub_font.render(f"Level {level}", 1, (255,255,255))
        WIN.blit(mode_label, (WIDTH - mode_label.get_width() - 20, 10))
        WIN.blit(lives_label, (22, 48))
        WIN.blit(total, (22, 100))
        WIN.blit(level_label, (22, 15))
        
        for enemy in enemies:
            enemy.draw(WIN)

        #GAME OVER SCREEN
        if lost:
            if multiplayer:
                main_font_b = pygame.font.Font("Game of Squids.ttf",75)
                sub_font_b = pygame.font.Font("Expansiva.ttf", 25)
                sub_font_2_b = pygame.font.Font("Expansiva.ttf", 30)
                text = main_font_b.render("Game Over",1,(255,255,255))
                text_4 = sub_font_2_b.render(f"P1 Score: {score}",1,(255,255,255))
                text_5 = sub_font_2_b.render(f"P2 Score: {score_2}",1,(255,255,255))
                text_2 = sub_font_b.render("[ENTER] PLAY AGAIN",1,(255,255,255))
                text_3 = sub_font_b.render("[BACKSPACE] RETURN TO MAIN MENU", 1, (255,255,255))
                WIN.blit(text, (WIDTH/2 - text.get_width()/2, 190))
                WIN.blit(text_4, (WIDTH/2 - text_4.get_width()/2, 320))
                WIN.blit(text_5, (WIDTH/2 - text_5.get_width()/2, 370))
                WIN.blit(text_2, (WIDTH/2 - text_2.get_width()/2, 450))
                WIN.blit(text_3, (WIDTH/2 - text_3.get_width()/2, 500))
                pygame.display.update()
                
                run = True
                while run:
                    for event in pygame.event.get():
                        if event.type == pygame.QUIT:
                            pygame.quit()
                            sys.exit()
                
                    keys = pygame.key.get_pressed() 
                    if keys[pygame.K_RETURN]:
                        start_screen()
                    if keys[pygame.K_BACKSPACE]:
                        run = False
                        main_menu()
            else:
                main_font_b = pygame.font.Font("Game of Squids.ttf",75)
                sub_font_b = pygame.font.Font("Expansiva.ttf", 25)
                sub_font_2_b = pygame.font.Font("Expansiva.ttf", 30)
                text = main_font_b.render("Game Over",1,(255,255,255))
                text_4 = sub_font_2_b.render(f"Score: {score}",1,(255,255,255))
                text_2 = sub_font_b.render("[ENTER] PLAY AGAIN",1,(255,255,255))
                text_3 = sub_font_b.render("[BACKSPACE] RETURN TO MAIN MENU", 1, (255,255,255))
                WIN.blit(text, (WIDTH/2 - text.get_width()/2, 230))
                WIN.blit(text_4, (WIDTH/2 - text_4.get_width()/2, 340))
                WIN.blit(text_2, (WIDTH/2 - text_2.get_width()/2, 390))
                WIN.blit(text_3, (WIDTH/2 - text_3.get_width()/2, 440))
                pygame.display.update()
                
                run = True
                while run:
                    for event in pygame.event.get():
                        if event.type == pygame.QUIT:
                            pygame.quit()
                            sys.exit()
                
                    keys = pygame.key.get_pressed() 
                    if keys[pygame.K_RETURN]:
                        start_screen()
                    if keys[pygame.K_BACKSPACE]:
                        run = False
                        main_menu()
        pygame.display.update()
    
    global player10, player20, player_laser_cooldown
    
    #PROGRAM
    while run:
        clock.tick(FPS)
        redraw_window()

        #MULTIPLAYER
        if multiplayer:
            if lives <= 0 or (player1.health <= 0 and player2.health <= 0):
                lost = True
                lost_count += 1
                
            if player1.health <= 0:
                player10 = False
            if player2.health <= 0:
                player20 = False
                   
            keys = pygame.key.get_pressed()
            if player10:
                if keys[pygame.K_a] and player1.x - player_vel > 0: # left
                    player1.x -= player_vel
                if keys[pygame.K_d] and player1.x + player_vel + player1.get_width() < WIDTH: # right
                    player1.x += player_vel
                if keys[pygame.K_w] and player1.y - player_vel > 0: # up
                    player1.y -= player_vel
                if keys[pygame.K_s] and player1.y + player_vel + player1.get_height() + 15 < HEIGHT: # down
                    player1.y += player_vel
                if keys[pygame.K_SPACE]:
                    player1.shoot()
            if player20:
                if keys[pygame.K_LEFT] and player2.x - player_vel > 0: # left
                    player2.x -= player_vel
                if keys[pygame.K_RIGHT] and player2.x + player_vel + player2.get_width() < WIDTH: # right
                    player2.x += player_vel
                if keys[pygame.K_UP] and player2.y - player_vel > 0: # up
                    player2.y -= player_vel
                if keys[pygame.K_DOWN] and player2.y + player_vel + player2.get_height() + 15 < HEIGHT: # down
                    player2.y += player_vel
                if keys[pygame.K_RCTRL]:
                    player2.shoot()
            
            if keys[pygame.K_p]:
                pause_screen()
        
            for enemy in enemies[:]:
                enemy.move(enemy_vel)
                enemy.move_lasers(2, player1)
                enemy.move_lasers(2, player2)

                if random.randrange(0, 2*60) == 1:
                    enemy.shoot()
                    if enemy.y > -25:
                        pygame.mixer.Sound.play(lasound_2)
                
                if player10:
                    if collide(enemy, player1):
                        player1.health -= 10
                        enemies.remove(enemy)
                        random_crash = random.choice(crash_list)
                        pygame.mixer.Sound.play(random_crash)
                if player20:
                    if collide(enemy, player2):
                        player2.health -= 10
                        enemies.remove(enemy)
                        random_crash = random.choice(crash_list)
                        pygame.mixer.Sound.play(random_crash)
                    
                if enemy.y + enemy.get_height() > HEIGHT:
                    lives -= 1
                    enemies.remove(enemy)
                    pygame.mixer.Sound.play(alarm)

            player1.move_lasers(-laser_vel, enemies)
            player2.move_lasers(-laser_vel, enemies)
                    
        #SINGLE PLAYER
        else:
            if lives <= 0 or player.health <= 0:
                lost = True
                lost_count += 1
                
            keys = pygame.key.get_pressed()
            if keys[pygame.K_a] and player.x - player_vel > 0: # left
                player.x -= player_vel
            if keys[pygame.K_d] and player.x + player_vel + player.get_width() < WIDTH: # right
                player.x += player_vel
            if keys[pygame.K_w] and player.y - player_vel > 0: # up
                player.y -= player_vel
            if keys[pygame.K_s] and player.y + player_vel + player.get_height() + 15 < HEIGHT: # down
                player.y += player_vel
            if keys[pygame.K_SPACE]:
                player.shoot()
            if keys[pygame.K_p]:
                pause_screen()

            for enemy in enemies[:]:
                enemy.move(enemy_vel)
                enemy.move_lasers(5, player)

                if random.randrange(0, 2*60) == 1:
                    enemy.shoot()
                    if enemy.y > -25:
                        pygame.mixer.Sound.play(lasound_2)

                if collide(enemy, player):
                    player.health -= 10
                    enemies.remove(enemy)
                    random_crash = random.choice(crash_list)
                    pygame.mixer.Sound.play(random_crash)
                    
                elif enemy.y + enemy.get_height() > HEIGHT:
                    lives -= 1
                    pygame.mixer.Sound.play(alarm)
                    enemies.remove(enemy)
                
            player.move_lasers(-laser_vel, enemies)
            

        #COMMON RUN #LEVEL INCREASE
        if len(enemies) == 0:
            level += 1
            if level != 1:
                pygame.mixer.Sound.play(level_up)
                player_laser_cooldown += 0.8
            wave_length += 5
            for i in range(wave_length):
                enemy = Enemy(random.randrange(50, WIDTH-50), random.randrange(-2000, -100), random.choice(["red", "blue", "green"]))
                #enemy = Enemy(THIS IS THE X, THIS IS THE Y, random.choice(["red", "blue", "green"]))
                enemies.append(enemy)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

                
                
#START SCREEN
def start_screen():
    run = True
    while run:
        global player10,player20
        WIN.blit(BG, (0,0))
        main_font = pygame.font.Font("Game of Squids.ttf",75)
        sub_font = pygame.font.Font("Expansiva.ttf", 25) 
        
        text = main_font.render("WARP",1,(255,255,255))
        text_2 = sub_font.render("[ENTER] INITIATE GAME",1,(255,255,255))
        text_3 = sub_font.render("[BACKSPACE] RETURN TO MAIN MENU", 1, (255,255,255))    
        WIN.blit(text, (WIDTH/2 - text.get_width()/2, 230))
        WIN.blit(text_2, (WIDTH/2 - text_2.get_width()/2, 340))
        WIN.blit(text_3, (WIDTH/2 - text_3.get_width()/2, 390))
        
        if multiplayer:
            player10 = True
            player20 = True
            player1 = Player(285, 550)
            player2 = Player2(945, 550)
            player1.draw(WIN)       
            player2.draw(WIN)        
        else:
            player = Player(605, 550)
            player.draw(WIN)    
        
        pygame.display.update()
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
                
            keys = pygame.key.get_pressed()
            if keys[pygame.K_BACKSPACE]:
                pygame.mixer.Sound.play(button_sound)
                run = False
                main_menu()
            if keys[pygame.K_RETURN]:
                pygame.mixer.Sound.play(warp)
                main()             
            

            
#PAUSE SCREEN
def pause_screen():
    main_font = pygame.font.Font("Game of Squids.ttf",75)
    sub_font = pygame.font.Font("Expansiva.ttf", 25) 
    
    text = main_font.render("Paused",1,(255,255,255))
    text_2 = sub_font.render("[ENTER] CONTINUE",1,(255,255,255))
    text_3 = sub_font.render("[BACKSPACE] RETURN TO MAIN MENU", 1, (255,255,255))
    WIN.blit(text, (WIDTH/2 - text.get_width()/2, 230))
    WIN.blit(text_2, (WIDTH/2 - text_2.get_width()/2, 340))
    WIN.blit(text_3, (WIDTH/2 - text_3.get_width()/2, 390))
    pygame.display.update()
    
    pause = True
    pygame.mixer.music.set_volume(0)
    while pause:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
                
        keys = pygame.key.get_pressed() 
        if keys[pygame.K_RETURN]:
            pygame.mixer.music.set_volume(0.5)
            pygame.mixer.Sound.play(button_sound)
            pause = False
        if keys[pygame.K_BACKSPACE]:
            pygame.mixer.music.set_volume(0.5)
            pygame.mixer.Sound.play(button_sound)
            main_menu()

            
            
#CONTROLS SCREEN
def controls_screen():
    run = True
    while run:
        CS = pygame.transform.scale(pygame.image.load("control_page.png"),(1280,720)) 
        WIN.blit(CS, (0,0))
        pygame.display.update()
        for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                
                keys = pygame.key.get_pressed()
                if keys[pygame.K_BACKSPACE]:
                    pygame.mixer.Sound.play(button_sound)
                    run = False
                if keys[pygame.K_c]:
                    pygame.mixer.Sound.play(button_sound)
                    run = False
                    credits()
def credits():
    run = True
    while run:
        CRS = pygame.transform.scale(pygame.image.load("credits.png"),(1280,720)) 
        WIN.blit(CRS, (0,0))
        pygame.display.update()
        for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                
                keys = pygame.key.get_pressed()
                if (keys[pygame.K_BACKSPACE]) or (keys[pygame.K_ESCAPE]):
                    pygame.mixer.Sound.play(button_sound)
                    run = False
                    controls_screen()
          
        
                
#BUTTON UI/UX SETUP
def button(msg,x,y,w,h,ic,ac,action=None):
    global multiplayer
    mouse = pygame.mouse.get_pos()
    click = pygame.mouse.get_pressed()
    
    main_font = pygame.font.Font("Game of Squids.ttf", 30)
    text = main_font.render(msg, 1, (255,255,255))

    if x+w > mouse[0] > x and y+h > mouse[1] > y:
        pygame.draw.rect(WIN, ac, pygame.Rect(x, y, w, h),  40, 30)
        if click[0] == 1 and action == "start":
            pygame.mixer.Sound.play(button_sound)
            multiplayer = False
            start_screen()
            WIN.blit(BG, (0,0))
        if click[0] == 1 and action == "multiplayer":
            pygame.mixer.Sound.play(button_sound)
            multiplayer = True
            start_screen()
            WIN.blit(BG, (0,0))
        if click[0] == 1 and action == "options":
            pygame.mixer.Sound.play(button_sound)
            controls_screen()
            main_menu()
        if click[0] == 1 and action == "exit_screen":
            pygame.mixer.Sound.play(button_sound)
            exit_screen() 
    else:
        pygame.draw.rect(WIN, ic, pygame.Rect(x, y, w, h),  40, 30)

    WIN.blit(text, (WIDTH/2 - text.get_width()/2, ((y+(h/2))-17)) )
    pygame.display.update()   
def button_b(msg,x,y,w,h,ic,ac,action=None):
    mouse = pygame.mouse.get_pos()
    click = pygame.mouse.get_pressed()
    
    main_font = pygame.font.Font("Game of Squids.ttf", 30)
    text = main_font.render("Yes", 1, (255,255,255))
    text_2 = main_font.render("No", 1, (255,255,255))

    if x+w > mouse[0] > x and y+h > mouse[1] > y:
        pygame.draw.rect(WIN, ac, pygame.Rect(x, y, w, h),  40, 30)
        if click[0] == 1 and action == "exit":
            pygame.mixer.Sound.play(button_sound)
            pygame.quit()
            sys.exit()
        if click[0] == 1 and action == "back":
            pygame.mixer.Sound.play(button_sound)
            main_menu()
    else:
        pygame.draw.rect(WIN, ic, pygame.Rect(x, y, w, h),  40, 30)

    WIN.blit(text, (390 , ((y+(h/2))-17)) )
    WIN.blit(text_2, (830 , ((y+(h/2))-17)) )
    pygame.display.update()
               
        

#EXIT SCREEN
def exit_screen():                                                               
    text_list = ["Leaving so soon?", "Why don't you stay a little longer?", "Good luck out there captain!", "Humanity depends on you, Sir.", "Don't forget us Capt.", "We'll wait for you, Sir."]
    exit_text = random.choice(text_list)
    
    WIN.blit(BG, (0,0))
    main_font = pygame.font.Font("Game of Squids.ttf",75)
    sub_font = pygame.font.Font("Expansiva.ttf",25)
    
    text = main_font.render("Quit Game?", 1, (255,255,255))
    text_2 = sub_font.render(exit_text, 1, (255,255,255))
    WIN.blit(text, (WIDTH/2 - text.get_width()/2, 230))
    WIN.blit(text_2, (WIDTH/2 - text_2.get_width()/2, 340)) 
    pygame.display.update()
    
    run = True
    while run:
        for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    run = False
                    pygame.quit()
                    sys.exit()
        
        button_b("Yes",340,440,180,80,(0,0,0),(90,90,90),"exit")
        button_b("No",760,440,180,80,(0,0,0),(90,90,90),"back")                
    pygame.quit()
    sys.exit()
          
        
                                                   
#MAIN MENU
def main_menu():
    text_list = ["Defender of Earth", "Faith of the galaxy lies in your hands...", "The last Defender", "Systems Ready", "Awaiting your command", "Let the Battle Commence!", "Your orders, Capt?"]
    welcome_text = random.choice(text_list)
    
    WIN.blit(BG, (0,0))
    main_font = pygame.font.Font("Game of Squids.ttf",80)
    sub_font = pygame.font.Font("Expansiva.ttf",20)
    
    text = main_font.render("Galactic Front", 1, (255,255,255))
    text_4 = sub_font.render(welcome_text, 1, (255,255,255))
    #text_2 = sub_font.render("Copyright 2021 PHUKET", 1, (255,255,255))
    #text_3 = sub_font.render("Yoputra, Lianto, Titoni, Lim, Evans, Antolin", 1, (255,255,255))
    WIN.blit(text, (WIDTH/2 - text.get_width()/2, 100))
    WIN.blit(text_4, (WIDTH/2 - text_4.get_width()/2, 85))
    #WIN.blit(text_2, (10,690))
    #WIN.blit(text_3, (WIDTH - text_3.get_width() - 10, 690))
    pygame.display.update()
    
    run = True
    while run:
        for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    run = False
                    pygame.quit()
                    sys.exit()
        
        button("Singleplayer",440,230,400,80,(0,0,0),(90,90,90),"start")
        button("Multiplayer",440,330,400,80,(0,0,0),(90,90,90),"multiplayer")
        button("Controls",520,430,240,80,(0,0,0),(90,90,90),"options")
        button("Exit",520,530,240,80,(0,0,0),(90,90,90),"exit_screen")           
    pygame.quit()
    sys.exit()



#LAUNCH
file = 'music.ogg'
pygame.mixer.music.load(file)
pygame.mixer.music.set_volume(0.5)
pygame.mixer.music.play(-1)
main_menu()

In [ ]:
#Keaton Yoputra
#Robby Lianto
#Jenifer Lim
#Erica Titoni
#Felix Evans
#Alex Handro Antolin